In [12]:
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import BayesianEstimator
from sklearn.model_selection import train_test_split
import pandas as pd

# Charger les données
data = pd.read_csv("breast-cancer.csv")

# Définir les noms des colonnes pour chaque attribut
data.columns = ['Class', 'Age', 'Menopause', 'Tumor_size', 'Inv_nodes', 'Node_caps', 'Deg_malig', 'Breast', 'Breast_quad', 'Irradiat']

# Diviser les données en ensembles d'entraînement et de test (80% / 20%)
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Créer un modèle bayésien avec la structure que vous avez déjà définie
model = BayesianNetwork([('Age', 'Menopause'),
                         ('Age', 'Deg_malig'),
                         ('Irradiat', 'Deg_malig'),
                         ('Irradiat', 'Breast'),
                         ('Menopause', 'Deg_malig'),
                         ('Tumor_size', 'Deg_malig'),
                         ('Tumor_size', 'Class'),
                         ('Inv_nodes', 'Class'),
                         ('Inv_nodes', 'Node_caps'),

                         ('Node_caps', 'Deg_malig'),
                         ('Tumor_size', 'Inv_nodes'),
                         ('Node_caps', 'Class'),
                         ('Breast', 'Breast_quad'),
                         ('Breast_quad', 'Node_caps'),
                         ('Class', 'Deg_malig')
                         ])

# Apprendre les paramètres du modèle avec l'algorithme EM sur l'ensemble d'entraînement
model.fit(train_data, estimator=BayesianEstimator, prior_type='BDeu', equivalent_sample_size=10)

# Afficher les distributions de probabilité conditionnelle apprises pour chaque nœud    
for cpd in model.get_cpds():
    print("CPD de {variable}:".format(variable=cpd.variable))
    print(cpd)

# Vous pouvez maintenant utiliser le modèle entraîné pour l'inférence et le diagnostic médical sur l'ensemble de test


CPD de Age:
+------------+-----------+
| Age(30-39) | 0.121849  |
+------------+-----------+
| Age(40-49) | 0.323529  |
+------------+-----------+
| Age(50-59) | 0.340336  |
+------------+-----------+
| Age(60-69) | 0.184874  |
+------------+-----------+
| Age(70-79) | 0.0294118 |
+------------+-----------+
CPD de Menopause:
+--------------------+-----+---------------------+
| Age                | ... | Age(70-79)          |
+--------------------+-----+---------------------+
| Menopause(ge40)    | ... | 0.8095238095238094  |
+--------------------+-----+---------------------+
| Menopause(lt40)    | ... | 0.09523809523809522 |
+--------------------+-----+---------------------+
| Menopause(premeno) | ... | 0.09523809523809522 |
+--------------------+-----+---------------------+
CPD de Deg_malig:
+--------------+-----+--------------------------+
| Age          | ... | Age(70-79)               |
+--------------+-----+--------------------------+
| Class        | ... | Class(recurrence-events

In [13]:
# Prédictions sur l'ensemble de test
predictions = model.predict(test_data.drop(columns=['Class']))

# Comparaison avec les étiquettes réelles
correct_predictions = (predictions['Class'] == test_data['Class']).sum()
total_samples = len(test_data)
accuracy = correct_predictions / total_samples

print(f"Précision sur l'ensemble de test : {accuracy}")


 14%|█▍        | 8/57 [00:04<00:30,  1.59it/s]

KeyboardInterrupt: 